## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками с помощью LDA в gensim и NMF в sklearn**.


1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases,  можно также попробовать другие способы построить нграммы); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) повторите пункт 4 на преобразованном корпусе;

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

In [1]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

stops = set(stopwords.words('russian'))
stops |= set(stopwords.words('english'))
morph = MorphAnalyzer()

## Данные

In [2]:
stops = set(stopwords.words('russian')) | {'gt',}
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def normalize(words):
    norm_words = [morph.parse(word)[0].normal_form for word in words if len(set(word)) > 1]
    return norm_words

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

## ДЗ начинаю отсюда:)

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - pip install pymorphy2[fast], можно использовать какой-то другой токенизатор);

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases, можно также попробовать другие способы построить нграммы);

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [132]:
%%time
texts = open('/Users/alinashaymardanova/Downloads/habr_texts.txt').read().splitlines()
texts = [tokenize(remove_tags(text.lower())) for text in texts]

CPU times: user 4 s, sys: 1.09 s, total: 5.09 s
Wall time: 6.49 s


In [133]:
%%time
texts = open('/Users/alinashaymardanova/Downloads/habr_texts.txt').read().splitlines()
texts = [normalize(tokenize(text.lower())) for text in texts]

CPU times: user 2min 17s, sys: 1.5 s, total: 2min 19s
Wall time: 2min 19s


In [134]:
%%time
texts = open('/Users/alinashaymardanova/Downloads/habr_texts.txt').read().splitlines()
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

CPU times: user 9.54 s, sys: 761 ms, total: 10.3 s
Wall time: 10.7 s


### Добавим в список стоп-слов lt, будем использовать mysem (вообще, опираясь на свою практику, могку сказать, что в большинстве случаев он работает лучше)

In [135]:
russian_stopwords = set(stopwords.words('russian')) | {'gt',} | {'lt',}
mystem = Mystem()

def preprocessing(text):
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub('\s{2,}', '', text)
    text = re.sub('[^a-zA-ZА-Яа-я]', ' ', text)
    tokens = mystem.lemmatize(text)
    tokens = [token for token in tokens if token not in russian_stopwords \
              and token != ' ' and token.strip() not in (punctuation + '«»—…“”*№–')]
    
    return tokens

In [136]:
texts = open('/Users/alinashaymardanova/Downloads/habr_texts.txt').read().splitlines()

In [137]:
texts = [preprocessing(remove_tags(text.lower())) for text in texts]

### Тематическое моделирование в gensim

Для моделей нужно сделать словарь.

In [51]:
dictinary = gensim.corpora.Dictionary(texts)

In [52]:
dictinary.filter_extremes(no_above=0.3)
dictinary.compactify()

In [53]:
print(dictinary)

Dictionary(24527 unique tokens: ['a', 'address', 'api', 'architecture', 'argumentnullexception']...)


Преобразуем наши тексты в мешки слов. 

In [130]:
corpus = [dictinary.doc2bow(text) for text in texts]

In [66]:
# ?gensim.models.LdaMulticore

In [55]:
%%time

#lda = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, passes=5, eta='auto', iterations=10, workers=8)
lda = gensim.models.LdaModel(corpus, id2word=dictinary, passes=10, num_topics=10, eta='auto', random_state=42)

CPU times: user 1min 4s, sys: 1.02 s, total: 1min 5s
Wall time: 1min 5s


In [60]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [61]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [62]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [63]:
coherence_model_lda.get_coherence()

0.63765170056602671

## N-граммы

In [138]:
# для нграммов
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

In [139]:
dictinary = gensim.corpora.Dictionary(ngrammed_texts)

In [140]:
dictinary.filter_extremes(no_above=0.3)
dictinary.compactify()

In [141]:
corpus = [dictinary.doc2bow(text) for text in ngrammed_texts]

In [45]:
%%time
#lda = gensim.models.LdaMulticore(corpus, id2word=dictinary, passes=5, eta='auto', iterations=10) 
lda = gensim.models.LdaModel(corpus, id2word=dictinary, passes=10, num_topics=10, eta='auto', random_state=42)

CPU times: user 50.1 s, sys: 943 ms, total: 51 s
Wall time: 50.9 s


In [142]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [146]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [ ]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [50]:
coherence_model_lda.get_coherence()

0.38807584882283669

###  Для самой хорошей модели в отдельной ячейке напечатаем 3 хороших темы;

In [183]:
topics[4]

0         объект
1       значение
2        элемент
3           блок
4          метод
5       алгоритм
6          точка
7         строка
8    изображение
9       текстура
Name: Тема 04, dtype: object

In [184]:
topics[18]

0         пациент
1            мозг
2          клетка
3          ученый
4     заболевание
5    исследование
6         болезнь
7            врач
8         лечение
9             ген
Name: Тема 18, dtype: object

In [185]:
topics[20]

0                  диск
1                сервер
2           виртуальный
3                память
4             резервный
5                машина
6                   ssd
7    производительность
8             процессор
9                    вм
Name: Тема 20, dtype: object

Кажется, что выглядит неплохо:)

### TfidfModel

In [124]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary)
corpus = tfidf[corpus]

In [125]:
lda = gensim.models.LdaModel(corpus, id2word=dictinary, passes=10, num_topics=100, eta='auto', random_state=42)

In [126]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [127]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [128]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

Tfidf не улучшил качество работы моделей, так как в топ попадает много слов, принадлежащих только одной узкой категории (названия функций, переменных и др.)

### Разложение матриц в sklearn

In [154]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [155]:
stexts = [' '.join(text) for text in texts]

In [156]:
vectorizer = TfidfVectorizer(max_features=25000, min_df=5, max_df=0.3, lowercase=False)
X = vectorizer.fit_transform(stexts)

In [157]:
model = NMF(n_components=30)

In [158]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [178]:
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(30):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Тема ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [179]:
get_nmf_topics(model, 10)

,Тема 01,Тема 02,Тема 03,Тема 04,Тема 05,Тема 06,Тема 07,Тема 08,Тема 09,Тема 10,...,Тема 21,Тема 22,Тема 23,Тема 24,Тема 25,Тема 26,Тема 27,Тема 28,Тема 29,Тема 30
0,продукт,if,игра,объект,файл,android,устройство,css,космический,атака,...,печать,центр,public,бот,сайт,звук,камера,товар,this,российский
1,клиент,int,игрок,значение,php,google,смартфон,js,спутник,безопасность,...,принтер,дата,string,telegram,реклама,сигнал,видео,скидка,react,страна
2,сотрудник,amp,игровой,элемент,http,ios,аккумулятор,javascript,орбита,уязвимость,...,станок,инфраструктура,void,сообщение,страница,усилитель,vr,магазин,div,россия
3,бизнес,return,играть,блок,docker,мобильный,телефон,react,ракета,злоумышленник,...,мм,услуга,new,bot,контент,частота,движение,цена,props,налог
4,менеджер,count,vr,метод,сервер,app,usb,angular,марс,пароль,...,материал,облачный,class,чат,браузер,наушник,регистратор,распродажа,компонент,рубль
5,заказчик,std,персонаж,алгоритм,sudo,layout,датчик,веб,аппарат,защита,...,печатать,оборудование,класс,api,клиент,искажение,реальность,покупатель,function,закон
6,crm,null,unity,точка,nginx,устройство,дисплей,vue,планета,устройство,...,производство,облако,private,message,домен,звуковой,blackvue,пятница,const,рынок
7,программист,const,steam,строка,скрипт,swift,ноутбук,html,земля,вредоносный,...,деталь,сервис,name,мессенджер,рекламный,музыка,виртуальный,покупка,return,доход
8,опыт,char,движок,изображение,name,play,корпус,браузер,луна,сеть,...,изделие,цод,return,канал,google,акустика,кадр,заказ,dom,налоговый
9,тестирование,else,жанр,текстура,file,view,батарея,es,станция,доступ,...,металл,провайдер,var,телегр,сервер,акустический,ip,акция,class,ооо


In [91]:
model.reconstruction_err_

60.473381463715747

А если взять другие параметры?

In [101]:
nvectorizer = TfidfVectorizer(max_features=30000, min_df=5, max_df=0.5)
nX = nvectorizer.fit_transform(stexts)

In [102]:
nmodel = NMF(n_components=30)
nmodel.fit(nX)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [103]:
nmodel.reconstruction_err_

60.130540306134954

In [96]:
nvectorizer = TfidfVectorizer(max_features=20000, min_df=5, max_df=0.5)

In [97]:
nX = nvectorizer.fit_transform(stexts)

In [98]:
nmodel = NMF(n_components=30)

In [99]:
nmodel.fit(nX)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [100]:
nmodel.reconstruction_err_

60.076327001679907

In [92]:
nvectorizer = TfidfVectorizer(max_features=10000, min_df=5, max_df=0.5)
nX = nvectorizer.fit_transform(stexts)

In [93]:
nmodel = NMF(n_components=30)

In [94]:
nmodel.fit(nX)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [95]:
nmodel.reconstruction_err_

59.63107858531545

Получаем, что LDA показывает себя хорошо, так как видно, что он выбирает хорошие ключевые слова. Однако насмотря на это, всё равно в итоговыую выборку попадает очень много мусора и не ключевых слов. При NMF шума меньше. Из-за этого, в LDA сложно выбрать хорошие примеры:( Таким образом, NMF работает лучше:)